<a id='1'></a>
<p style = "background-color : #79d1ff; border-radius: 5px 5px; padding : 10px"><strong>

# Обзор

<p style = "background-color : #79d1ff; border-radius: 5px 5px; padding : 10px"><strong>

**Стратегия это - функция для открытия и закрытия позиций созданная из индикатора**.  Например, если у вас есть индикатор, отображающий пересечение скользящих средних, вы можете превратить его в стратегию, добавив команды strategy.entry для входа и strategy.close для выхода из позиций. 
т.е. по сути это индикатор или комплекс индикаторов заключенной в одной функции.

<p style = "background-color : #afeeee; border-radius: 5px 5px; padding : 10px"><strong>
<span style="color:#000">Импортируем библиотеки используемые в работе:</span>

In [72]:
# Расчёты
import numpy as np
import pandas as pd

# Визуализация
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# Для распечатки в цвете:
from termcolor import colored

import re
# import nltk
# from nltk.stem import WordNetLemmatizer
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize
from collections import Counter

<p style = "background-color : #afeeee; border-radius: 5px 5px; padding : 10px"><strong>
<span style="color:#000">Функции:</span>

<p style = "background-color : #95f59ff2; border-radius: 5px 5px; padding : 5px"><strong>
<span style="color:#000">Графики</span>

In [73]:
def build_graph(library, chart_type, data_frame, x, y, height, width, title, xaxis_title, yaxis_title, legend = None, color = None):
    """Функция по построению графиков

    Args:
        library: Библиотека построения
        chart_type: Тип графика
        data_frame: Датафрейм из которого будет читаться данные
        x: Ось x
        y: Ось y
        height: Высота графика
        width: Ширина графика
        title: Название графика
        xaxis_title: Название оси x
        yaxis_title: Название оси y
        legend: Название легенды
        color: Цвет колонок
    Raises:
        ValueError: Ошибка вызова графика
    """
    if library == 'plt':
        plt.figure(figsize=(height, width))
        plt.title(title)
        if chart_type == 'box':
            sns.boxplot(data=data_frame, x=x, y=y)
        elif chart_type == 'bar':
            sns.barplot(data=data_frame, x=x, y=y)
        else:
            raise ValueError("Неподдерживаемый тип графика. Допустимые значения chart_type: 'box', 'bar'")
        plt.xlabel(xaxis_title)
        plt.ylabel(yaxis_title)
        plt.show()
    # plotly
    elif library == 'px':
        if chart_type == 'box':
            fig = px.box(data_frame=data_frame, x=x, y=y, color=color, title=title)
        elif chart_type == 'bar':
            fig = px.bar(data_frame=data_frame, x=x, y=y, color=color, title=title)
        else:
            raise ValueError("Неподдерживаемый тип графика. Допустимые значения chart_type: 'box', 'bar'")
        
        fig.update_layout(
            title=dict(x=0.5, xanchor="center"),
            legend_title=legend,
            xaxis_title=xaxis_title,
            yaxis_title=yaxis_title,
            margin=dict(l=0, r=0, t=30, b=0),
            title_x=0.5
        )
        fig.show()
    else:
        raise ValueError("Неподдерживаемая библиотека. Допустимые значения library: 'px', 'plt'")


In [74]:
def pie_graph(data, title_text):
    # Построение круговой диаграммы
    fig = go.Figure(
        data=[go.Pie(       
            values=data.values, 
            labels=data.index, 
            pull=[0.1]*len(data)
        )]
    )


    fig.update_traces(
        hoverinfo='label+value',  # Добавляем информацию при наведении
        textinfo='percent',
        textfont_size=15,
        marker=dict(line=dict(color='#000000', width=2))
    )

    # Устанавливаем название графика и размеры текста:
    fig.update_layout(
        title_text=title_text,
        title_font_size=20,
        legend_font_size=20,
        width=800,
        height=600
    )

    fig.show()

In [75]:
def gist_graph(df, column, title):    
    # Вычисляем моду, это понадобится для выделение моды на графике
    sharpe_mode = df[column].mode()[0]

    fig = px.histogram(
        df,   
        x = column,
        marginal='box',
        nbins=200,
        width=1000,
        height=500,
        color_discrete_sequence=['Orangered']
    )

    # Устанавливаем название графика, размеры текста и наименование осей:
    fig.update_layout(
        title={
            'text': title,
            'y':0.97,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'
            },
        title_font={
            'size':18,
            'color':"black"
            },
        xaxis_title='Размер коэффициента',
        yaxis_title='Количество вхождений'
    )

    # Добавляем обводку колонок
    fig.update_traces(
        hoverinfo="all",
        hovertemplate="Диапазон размера коэффициента: %{x}",
        marker_line_color='black',   # Цвет линии колонки
        marker_line_width=1.0,       # Толщина линии колонки
        opacity=0.9            # Прозрачность колонки
    )

    # Устанавливаем дополнительную линию моды на графике:
    fig.add_vline(
        x=sharpe_mode,
        line_width=2, 
        line_color="black", 
        annotation_text=f"Мода распределения: {sharpe_mode}",
        annotation_position='bottom left'
    )

    fig.update_xaxes(showspikes=True)
    fig.update_yaxes(showspikes=True)
    fig.show()

<p style = "background-color : #95f59ff2; border-radius: 5px 5px; padding : 5px"><strong>
<span style="color:#000">Работа с текстом</span>

In [76]:
# Функция для очистки текста
def clean_text(text):
    # Удаление эмодзи и спецсимволов, кроме пробелов
    text = re.sub(r'[^\w\s]', '', str(text))
    # Удаление лишних пробелов
    text = re.sub(r'\s+', ' ', text).strip().lower()
    return text

In [77]:
def classify_tags(row, tags_dict):
    """Функция для классификации тегов на основе нового словаря."""
    text_to_analyze = f"{str(row['strategy_description'])} {str(row['tags'])}"
    classifications = []

    for category, keywords in tags_dict.items():
        if any(re.search(r'\b' + re.escape(keyword) + r'\b', text_to_analyze) for keyword in keywords):
            classifications.append(category)

    # Если найдено более одного тега, возвращаем 'mixed' УБРАТЬ ЭТУ СТРОКУ ЕСЛИ НАМ НЕОБХОДИМЫ ВСЕ ВХОЖДЕНИЯ
    if len(classifications) > 1:
        return 'combined strategy'
    
    return classifications[0] if classifications else 'No classification'

In [78]:
def check_keywords(df, tags):
    """Функция для проверки наличия ключевых слов в датафрейме."""
    # Объединяем описание и теги
    df['combined'] = (df['strategy_description'].fillna('') + ' ' + df['tags'].fillna('')).str.lower()
    
    # Функция для обработки каждой строки
    def process_text(text):
        matched_tags = []
        for keyword in tags:
            if keyword in text:
                matched_tags.append(keyword)
        return matched_tags

    # Применяем функцию к объединенному тексту
    df['type_tags'] = df['combined'].apply(process_text)
    return df.drop(columns=['combined'])

In [79]:
def convert_to_numeric(val):
    # Заменяем символы и удаляем пробелы
    val = val.replace('−', '-')  # Заменяем минус на стандартный
    val = val.replace('+', '')    # Удаляем знак плюс
    val = val.replace(',', '')     # Удаляем запятые
    return pd.to_numeric(val, errors='coerce')  # для преобразования строки в числовой формат, при этом errors='coerce' заменяет некорректные значения на NaN.

<p style = "background-color : #95f59ff2; border-radius: 5px 5px; padding : 5px"><strong>
<span style="color:#000">Создание датафреймов</span>

In [80]:
def process_tags(df, tags_dict):
    """Функция по созданию датафреймов согласно списку тегов."""
    tag_counts_results = {}

    for tag_name, keywords in tags_dict.items():
        # Проверяем ключевые слова из ранее сделанной функции
        filtered_df = check_keywords(df, keywords)
        # Убираем строки с пустыми классификациями
        filtered_df = filtered_df[filtered_df['type_tags'].apply(lambda x: x != [])]
        # Сохраняем только нужные столбцы
        filtered_df = filtered_df[['title', 'type_tags']]
        
        # Разбиваем списки на отдельные строки
        exploded_tags = filtered_df.explode('type_tags')
        
        # Подсчитываем количество вхождений каждого тега
        tag_counts = exploded_tags['type_tags'].value_counts().reset_index()
        tag_counts.columns = ['tag', 'count']  # Переименовываем колонки
        
        # Сортируем по количеству вхождений
        tag_counts = tag_counts.sort_values(by='count', ascending=False)
        
        # Сохраняем результат в словаре
        tag_counts_results[tag_name] = tag_counts

    return tag_counts_results


<a id='1'></a>
<p style = "background-color : #79d1ff; border-radius: 5px 5px; padding : 10px"><strong>

# Обзор колонок датасета

<p style = "background-color : #79d1ff; border-radius: 5px 5px; padding : 10px"><strong>

<div class="alert alert-info">

##### **Overview (Обзор)**
- <span style="color:#af0400">**title**</span> -  <span style="color:#02020A">**Название стратегии**</span>;
- <span style="color:#af0400">**strategy_description**</span> - <span style="color:#02020A">**Описание стратегии**</span>;
- <span style="color:#af0400">**tags**</span> - <span style="color:#02020A">**Теги к стратегии**</span>;

##### **Performance (Отчет о стратегии)**
- <span style="color:#af0400">**Open P&L**</span> - <span style="color:#02020A">**Анализ сделок**</span> (Прибыль или убыток по текущей открытой позиции. Если позиция не открыта, возвращается значение N/A)</span>;
- <span style="color:#af0400">**Net profit**</span> - <span style="color:#02020A">**Чистая прибыль**</span> (Значение представляет собой сумму всех значений из столбца «Прибыль»);
- <span style="color:#af0400">**Gross profit**</span> - <span style="color:#02020A">**Валовая прибыль**</span> (Общая прибыль по всем прибыльным сделкам, генерируемая стратегией);
- <span style="color:#af0400">**Gross loss**</span> - <span style="color:#02020A">**Валовый убыток**</span> (Общие убытки по всем убыточным сделкам, совершенным по стратегии. Анализ и сокращение торговых убытков — чрезвычайно важная часть анализа торговой стратегии. Именно поэтому эта характеристика стратегии является наиболее важной. Следует отметить, что чистая прибыль увеличивается не только при увеличении валовой прибыли, но и при сокращении валовых убытков);
- <span style="color:#af0400">**Commission paid**</span> - <span style="color:#02020A">**Уплаченная комиссия**</span>;
- <span style="color:#af0400">**Buy & hold returntact**</span> - <span style="color:#02020A">**Возврат средств при покупке и удержании**</span> (Доходность, полученная в случае, если все средства (начальный капитал) были использованы для покупки ценной бумаги при открытии первой сделки и позиция удерживалась в течение всего тестового периода);
- <span style="color:#af0400">**Max equity run-up**</span> - <span style="color:#02020A">**Максимальный прирост собственного капитала**</span> (Отображает максимальный выигрыш, то есть максимально возможный выигрыш, который стратегия могла бы получить по всем совершённым сделкам);
- <span style="color:#af0400">**Max equity drawdown**</span> - <span style="color:#02020A">**Максимальная просадка собственного капитала**</span> (Отображает наибольшую просадку по убыткам, то есть максимально возможный убыток, который могла бы понести стратегия по всем совершённым сделкам.);
- <span style="color:#af0400">**Max contracts held**</span> - <span style="color:#02020A">**Максимальное количество заключенных контрактов**</span> (Максимальное количество контрактов, заключенных одновременно.).


##### **Trades analysis (Анализ сделок)**
- <span style="color:#af0400">**Total trades**</span> - <span style="color:#02020A">**Общее количество сделок**</span> (как выигрышных, так и убыточных);
- <span style="color:#af0400">**Total open trades**</span> - <span style="color:#02020A">**Общее количество открытых сделок**</span> (в данный момент);
- <span style="color:#af0400">**Winning trades**</span> - <span style="color:#02020A">**Выигрышные сделки**</span>;
- <span style="color:#af0400">**Losing trades**</span> - <span style="color:#02020A">**Убыточные сделки**</span>;
- <span style="color:#af0400">**Percent profitable**</span> - <span style="color:#02020A">**Процент прибыльности**</span> (Процент прибыльных сделок, совершённых по стратегии. Рассчитывается путём деления количества прибыльных сделок на общее количество закрытых сделок, совершённых по стратегии. Процент прибыльных сделок сам по себе не является надёжным показателем);
- <span style="color:#af0400">**Avg P&L**</span> - <span style="color:#02020A">**Средний доход за год**</span> (Сумма денег, полученная или потерянная в результате средней сделки, заключённой по стратегии. Рассчитывается путём деления чистой прибыли на общее количество закрытых сделок. Важное значение, поскольку оно должно быть достаточно большим, чтобы покрыть комиссию и издержки, связанные с проскальзыванием, и при этом приносить прибыль);
- <span style="color:#af0400">**Avg winning trade**</span> - <span style="color:#02020A">**Средняя выигрышная сделка**</span> (Валовая прибыль, делённая на количество прибыльных сделок, совершённых по стратегии);
- <span style="color:#af0400">**Avg losing trade**</span> - <span style="color:#02020A">**Средняя убыточная сделка**</span> (Общий убыток, делённый на количество убыточных сделок, открытых по стратегии);
- <span style="color:#af0400">**Ratio avg win / avg loss**</span> - <span style="color:#02020A">**Соотношение avg выигрыш / avg проигрыш**</span> (Среднее значение того, сколько денежных единиц вы выигрываете на каждую потерянную единицу (в выбранной валюте). Это значение рассчитывается путём деления среднего количества выигрышных сделок на среднее количество проигрышных сделок. Само по себе это поле не является очень значимым, поскольку оно не учитывает соотношение количества выигрышных и проигрышных сделок, а стратегии могут иметь разные подходы к прибыльности);
- <span style="color:#af0400">**Largest winning trade**</span> - <span style="color:#02020A">**Самая крупная выигрышная сделка**</span> (Наибольшая денежная прибыль и наибольшая процентная прибыль. Эти показатели обычно относятся к одной и той же сделке, но могут относиться и к разным сделкам);
- <span style="color:#af0400">**Largest winning trade percent**</span> - <span style="color:#02020A">**Процент самой крупной выигрышной сделки**</span>;
- <span style="color:#af0400">**Largest losing trade**</span> - <span style="color:#02020A">**Самая убыточная сделка**</span>;
- <span style="color:#af0400">**Largest losing trade percent**</span> - <span style="color:#02020A">**Процент самой убыточной сделки**</span>(Наибольшая денежная потеря и наибольшая процентная потеря. Эти показатели обычно относятся к одной и той же сделке, но могут относиться и к разным сделкам);
- <span style="color:#af0400">**Avg # bars in trades**</span> - <span style="color:#02020A">**Среднее количество в баров сделок**</span>;
- <span style="color:#af0400">**Avg # bars in winning trades**</span> - <span style="color:#02020A">**Среднее количество в баров в выигрышных сделках**</span>;
- <span style="color:#af0400">**Avg # bars in losing trades**</span> - <span style="color:#02020A">**Среднее количество в баров убыточных сделках**</span>;
 
##### **Risk / performance ratios (Соотношения риска и доходности)**
- <span style="color:#af0400">**Sharpe ratio**</span> - <span style="color:#02020A">**Коэффициент Шарпа**</span> (Чем выше коэффициент Шарпа, тем более плавная кривая доходности. Для многих трейдеров плавная кривая доходности является важной целью);
- <span style="color:#af0400">**Sortino ratio**</span> - <span style="color:#02020A">**Коэффициент Сортино**</span> (Даёт более полное представление об эффективности портфеля с учётом риска, поскольку положительная волатильность считается преимуществом);
- <span style="color:#af0400">**Profit factor**</span> - <span style="color:#02020A">**Коэффициент прибыли**</span> (Сумма денег, которую торговая стратегия заработала на каждую единицу потерянных денег (в выбранной валюте). Это значение рассчитывается путём деления валовой прибыли на валовые убытки);
- <span style="color:#af0400">**Margin calls**</span> - <span style="color:#02020A">**Маржин-коллы**</span> (Общее количество маржин-коллов, сгенерированных стратегией);

##### **Properties (Свойства стратегий)**
- <span style="color:#af0400">**properties**</span> - <span style="color:#02020A">**Свойства**</span> (Содержит в себе диапазон торговых дат, Информации о символах, исходные данные стратегии и свойства стратегии);


<center><p style = "background-color : white; border-radius: 2px 2px; padding : 3px"><strong>
<span style="color:#000">Дополнительные материалы:</span>

<span style="color:#00FF00">**Коэффициент Шарпа**</span> — это метрика, которая оценивает эффективность инвестиций, учитывая их доходность с поправкой на риск. Он показывает, насколько хорошо актив или стратегия вознаграждает инвестора за принятый риск. Чем выше значение, тем лучше соотношение доходности и риска.
$$
\text{Sharpe Ratio} = \frac{R_p - R_f}{\sigma_p}
$$
Где:  
- $R_p$ — средняя доходность портфеля/стратегии.  
- $R_f$ — безрисковая ставка (например, доходность гособлигаций).  
- $\sigma_p$ — стандартное отклонение доходности портфеля (волатильность).

**Как интерпретировать коэффициент Шарпа?**

> $>2$ — Отлично: высокая эффективность.

> $>1$ — Хорошо: доходность компенсирует риск.

> $<1$ — Низкая эффективность: риск не оправдан доходностью. 



<span style="color:#00FF00">**Коэффициент Сортино**</span> - это метрика, оценивающая доходность инвестиций с учётом только негативного риска (downside risk). В отличие от коэффициента Шарпа, который учитывает общую волатильность (как рост, так и падение), Сортино фокусируется на риске убытков. Это делает его более подходящим для оценки стратегий, где важно контролировать просадки.
$$
\text{Sortino Ratio} = \frac{R_p - R_f}{\sigma_d}
$$

Где:  
- $R_p$ — средняя доходность портфеля.  
- $R_f$ — безрисковая ставка (например, доходность гособлигаций).  
- $\sigma_d$ — downside deviation (стандартное отклонение только тех доходностей, которые ниже целевого уровня, например, $R_f$).

**Как интерпретировать коэффициент Шарпа?**
> $>2$ — Отлично: Стратегия/актив приносит высокую доходность с минимальными просадками.

> $>1$ — Хорошо: Риск убытков оправдан доходностью.

> $<1$ — Слабый результат: Риск убытков слишком высок относительно доходности. 

> $<0$ - Опасный сигнал: Средняя доходность ниже безрисковой ставки. Инвестор теряет деньги с учётом риска

<span style="color:#00FF00">**Коэффициент прибыли**</span> - показывает соотношение общей прибыли к общим убыткам. Это ключевой показатель для оценки эффективности торговой стратегии.


$$
\text{Profit Factor} = \frac{\sum \text{Profit}}{\sum \text{Loss}}
$$

> $>2$ — Cтратегия генерирует вдвое больше прибыли, чем убытков

> $>1$ — Стратегия прибыльна.

> $=1$ — Безубыточность.

> $<1$ —  Убыточность.


<span style="color:#FF4500">**Сравнение Profit Factor, Коэффициента Шарпа и Коэффициента Сортино**</span>

| Параметр                | Profit Factor                  | Коэффициент Шарпа             | Коэффициент Сортино            |
|-------------------------|--------------------------------|--------------------------------|---------------------------------|
| **Что измеряет**         | Соотношение прибыли и убытков  | Доходность на единицу общего риска | Доходность на единицу downside-риска |
| **Формула**             | (Сумма прибыли) / (Сумма убытков) | (Rp - Rf) / σp                | (Rp - Rf) / σd                  |
| **Учитываемый риск**    | Не учитывает                   | Общая волатильность            | Только негативная волатильность |
| **Идеальное значение**  | ≥ 2.0                         | ≥ 1.5                          | ≥ 2.0                          |
| **Где применять**       | Оценка рентабельности стратегии | Сравнение с рыночными индексами | Контроль просадок              |
| **Пример расчета**      | Прибыль: $15k, Убытки: $5k → PF = 3.0 | Rp=12%, Rf=2%, σp=10% → (12-2)/10 = 1.0 | Rp=15%, Rf=2%, σd=5% → (15-2)/5 = 2.6 |
| **Плюсы**               | Простота интерпретации         | Универсальность                | Фокус на риск убытков           |
| **Минусы**              | Игнорирует размер и частоту сделок | Наказывает за позитивную волатильность | Зависит от выбора MAR*          |
| **Подходит для**        | Начинающих трейдеров           | Долгосрочных инвесторов        | Консервативных стратегий        |

*MAR (Minimum Acceptable Return) — минимально допустимая доходность.

---

### Краткие пояснения:
1. **Profit Factor**  
   - Быстрый "чек-ап" стратегии: показывает, во сколько раз прибыль превышает убытки.  
   - Пример: PF = 2 → на каждые $1 убытка приходится $2 прибыли.

2. **Коэффициент Шарпа**  
   - Универсальный стандарт для сравнения активов.  
   - Пример: Sharpe = 1.5 → доходность на 50% выше общего риска.

3. **Коэффициент Сортино**  
   - Идеален для стратегий с жестким стоп-лоссом.  
   - Пример: Sortino = 3 → доходность в 3 раза превышает риск просадок.



<span style="color:#00FF00">**Маржин-коллы**</span> - это требование брокера к трейдеру пополнить счет или закрыть часть позиций, когда уровень маржи (свободных средств) падает ниже допустимого минимума. Это происходит, если убытки по открытым позициям «съедают» залоговые средства. Игнорирование маржин-колла может привести к принудительному закрытию позиций брокером.

<div class="alert alert-info">
    
Читаем данный после парсинга

In [81]:
df = pd.read_csv('tradingview_strategies_full.csv')

In [82]:
df

,title,strategy_description,tags,Open P&L,Net profit,Gross profit,Gross loss,Commission paid,Buy & hold return,Max equity run-up,...,Largest losing trade,Largest losing trade percent,Avg # bars in trades,Avg # bars in winning trades,Avg # bars in losing trades,Sharpe ratio,Sortino ratio,Profit factor,Margin calls,properties
0,Daily Breakout + Daily Shadow By Rouro,This script is a Pine v5 strategy designed to ...,"{'statistics', 'Candlestick analysis', 'Chart ...",0,"+8,375.00","9,973.00","1,598.00",0,"+117,304.17","8,375.00",...,"1,598.00",0.51%,50,63,1,0.203,0.453,6.241,0.0,"{'Trading range': 'Mar 10, 2025, 23:05 — May 0..."
1,Order Block Strategy,Strategy OverviewKey FeaturesOrder Block Detec...,"{'Pivot points and levels', 'Pine utilities', ...","−1,453.09","+9,783.49","9,783.49",0.00001,0,−72.38,"10,667.89",...,0.000004,3.49%,"1,152","1,348",71,0.616,9.392,"1,426,164,099.778",0.0,"{'Trading range': 'Oct 01, 2024, 15:30 — May 0..."
2,EXODUS,EXODUS by (D...,"{'Volume', 'Volatility', 'Oscillators'}",0,"+14,293.23","68,836.40","54,543.18","1,672.77","+5,593,815.28","20,726.51",...,"1,767.68",4.71%,4,5,3,−1.136,−0.782,1.262,0.0,"{'Trading range': 'Jan 30, 2020, 11:00 — May 0..."
3,External Signals Strategy Tester v5,External Signals Strategy Tester v5 – User Gui...,"{'educational', 'Bollinger Bands (BB)', 'Bands...",0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'Trading range': ' — ', 'Backtesting range': ..."
4,Guaranteed Entry Strategy (Every 5 Bars),"//version=5strategy(""Guaranteed Entry Strategy...",{'regressions'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{}
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
954,rt maax EMA cross strategy,this just sample of our strategies we publishe...,"{'BTC', 'sma', 'trend', 'Trend Analysis', 'For...","+39,115.55","+1,066,063.89","1,108,975.30","42,911.41","12,611.77","+9,960,471.92","1,106,625.50",...,"11,085.32",10.95%,766,952,282,0.024,2.104,25.843,0.0,"{'Trading range': 'Feb 01, 1933 — Oct 01, 2024..."
955,SPX Fair Value Strategy Ultimate,This is a strategy using the SPX Fair Value de...,{'Oscillators'},0,"+469,888.27","469,888.27",0,0,"−167,001.45","469,888.27",...,NaN,NaN,NaN,NaN,NaN,0.607,28.369,NaN,NaN,"{'Trading range': 'Jan 03, 2022 — Nov 22, 2022..."
956,Multi Trend Cross Strategy Template,Today I am sharing with the community trend cr...,"{'regressions', 'strategy', 'Trend Analysis', ...",0,"+28,127,665.94","31,327,842.85","3,200,176.91","125,781.93","+13,648,918.30","28,133,021.71",...,"1,242,107.46",15.16%,151,262,40,0.137,1.885,9.789,0.0,"{'Trading range': 'Nov 03, 2010 — Oct 26, 2022..."
957,BOLLY Bands,This is a strategy using Bollinger Bands. The ...,"{'bollingerbandstrategy', 'Bollinger Bands (BB...",0,"+7,998.34","11,559.50","3,561.16","1,448.00","+3,159.02","8,014.34",...,224.51,0.22%,57,51,72,1.367,443.519,3.246,0.0,"{'Trading range': 'Oct 17, 2022, 00:35 — Oct 2..."


<div class="alert alert-info">
    
Разберемся, что с пропусками

<div class="alert alert-danger">
    
Пропуска есть, но обусловимся тем, что у каждой стратегии может быть индивидуальные показатели, и внесение средних, медианных, категориальных данных может ухудшить результаты на обучении. Поэтому мы просто почистим строки где есть пропуска. В случае, если не итоги будут не корректными вернемся на этап пропусков и попробуем их заполнить.

In [83]:
# Почистим от пропусков
df.dropna(inplace=True)

--------------------------------

In [84]:
# Сохраним почищенный датафрейм, потому что наверняка его разрушим
data = df.copy() 

<p style = "background-color : #95f59ff2; border-radius: 5px 5px; padding : 10px"><strong>
<span style="color:#000">Приведем текстовые колонки в нормальный вид:</span>

In [85]:
# Применим функцию
# Применяем функцию к каждой колонке отдельно
df['title'] = df['title'].apply(clean_text)
df['strategy_description'] = df['strategy_description'].apply(clean_text)
df['tags'] = df['tags'].apply(clean_text)  # Если 'tags' - это строка, иначе нужно будет обработать по-другому


<div class='alert alert-info'>
Разберемся с тем, как нам разделять стратегии

##### **По временным интервалам** 

- <span style="color:#00FA9A">**Краткосрочные (скальпинг, внутридневная торговля)**</span>
- <span style="color:#00FA9A">**Среднесрочные (свинг-трейдинг)**</span>
- <span style="color:#00FA9A">**Долгосрочные (позиционная торговля)**</span>

##### **По методам анализа**
- <span style="color:#00FA9A">**Стратегии на основе технического анализа (индикаторы, паттерны)**</span>
- <span style="color:#00FA9A">**Стратегии на основе объема**</span>
- <span style="color:#00FA9A">**Стратегии на основе ценового действия (Price Action)**</span>
- <span style="color:#00FA9A">**Комбинированные стратегии**</span>

##### **По применяемым индикаторам**
- <span style="color:#00FA9A">**Трендовые (MA, MACD, ADX)**</span>
- <span style="color:#00FA9A">**Осцилляторы (RSI, Stochastic)**</span>
- <span style="color:#00FA9A">**Волатильность (Bollinger Bands, ATR)**</span>
- <span style="color:#00FA9A">**Объемные (OBV, Money Flow Index)**</span>

##### **По механизму входа/выхода**
- <span style="color:#00FA9A">**Пробойные стратегии**</span>
- <span style="color:#00FA9A">**Стратегии возврата к среднему**</span>
- <span style="color:#00FA9A">**Стратегии следования за трендом**</span>
- <span style="color:#00FA9A">**Контртрендовые стратегии**</span>
##### **По типу рынк**
- <span style="color:#00FA9A">**Для акций**</span>
- <span style="color:#00FA9A">**Для криптовалют**</span>
- <span style="color:#00FA9A">**Для фьючерсов**</span>
- <span style="color:#00FA9A">**Для форекс**</span>

<a id='1'></a>
<p style = "background-color : #79d1ff; border-radius: 5px 5px; padding : 10px"><strong>

# Описание стратегий

<p style = "background-color : #79d1ff; border-radius: 5px 5px; padding : 10px"><strong>

<div class='alert alert-info'>

Внутри каждой стратегии находятся дополнительный подтипы, разделения на внутренние подтипы не увенчались успехом и получилось много различных и сложных комбинаций разделений стратегий. В связи с этим мы оставили только основные типы, без внутренних разделений и **создали словарь для классификации стратегий со всеми возможными тегами и словам**.

*Ранее этот словарь был наполнен большим количеством тегов, но после анализа и исключений неопределенных слов приобрел данный вид*

In [86]:
# Новый словарь тегов
tags = {
    "time": ['position', 'momentum', 'shortterm', 'trendfollowing', 'swing', 'longterm',
             'intraday', 'mediumterm', 'scalp', 'invest', 'highfrequency', 'daytrade', 
             'multiday', 'trendfollow'],
    'analysis_Method': ['indicator', 'atr', 'technical', 'volume', 'candlestick', 'liquidity',
                        'fundamental', 'fibonacci', 'vwap', 'supportresistance', 'engulfing', 
                        'earnings', 'obv', 'priceaction', 'harmonic', 'sr', 'cvd', 'pe', 
                        'valuation', 'financials'],
    'indicators': ['ema', 'atr', 'rsi', 'sma', 'ma', 'macd', 'bollinger', 'oscillator',
                   'supertrend', 'adx', 'stochastic', 'breadth', 'fibonacci', 'cci', 
                   'keltner', 'ichimoku', 'distribution', 'roc', 'decline', 'obv', 
                   'sar', 'advance', 'accumulation', 'zlsma', 'mfi', 'dpo', 
                   'movingaverage', 'chaikin'],
    'mechanism': ['reversal', 'oversold', 'overbought', 'breakout', 'divergence', 
                       'reversion', 'countertrend', 'movingaveragecrossover', 'channelbreak', 
                       'trendfollow'],
    'market': ['future', 'equity', 'crypto', 'forex', 'bitcoin', 'currency', 
                    'fx', 'contango', 'backwardation', 'liquidation', 'hodl', 'altcoin']
}

In [87]:
# Выполним классификацию стратегий на основе тегов
df['classification'] = df.apply(lambda x: classify_tags(x, tags), axis=1)

In [88]:
# Узнаем сколько в каждом классе находится стартегий
cl_df = df['classification'].value_counts()
# Строим график
pie_graph(cl_df, 'Соотношение типов стратегий')

<div class="alert alert-success">
    
**Как мы видим из графика чаще всего типы стартегий комбинируются, и сложно выделить стратегию в определенный класс.**

*Данное утверждение опирается исключительно из информации по описанию стратегии.Ну и в целом, логично комбинировать разные типы, чтобы достичь лучшего результата.*

<div class='alert alert-info'>
Теперь разберем вхождение терминов по отдельным направлениям стратегий:

In [89]:
# Вызовем раннюю написанную функцию, для разделения
tag_counts_results = process_tags(df, tags)

# Создадим новые датафреймы для удобства по каждому классу
df_time = tag_counts_results['time']
df_method = tag_counts_results['analysis_Method']
df_indicator = tag_counts_results['indicators']
df_mechanism = tag_counts_results['mechanism']
df_market = tag_counts_results['market']

In [90]:
# Список новых датафреймов
list_df = [df_time, df_indicator, df_market, df_mechanism, df_method]
# Название графиков
title_name = ['time_tags', 'indicator_tags', 'market_tags', 'mechanism_tags', 'method_tags']
# Строим графики по новым датафреймам:
for list, name in zip(list_df, title_name):
    build_graph('px', 'bar', list, 'tag', 'count', 20, 8, name, 'Теги', 'Количество', color='tag')

<div class="alert alert-success">
    
Из созданных датафреймов получили визуализацию вхождения каждых тегов, какие теги чаще входят в ту или иную стратегию, это нам в будущем поможет определиться с лучшей стратегией. Мы определим, какие стратегии наиболее выгодные с лучшей прибылью, и к чему какая применяется, проанализируем эти эффективные стратегии на вхождение тегов, в большой части на вхождение индикаторов, определим, какие индикаторы и методы чаще используются в этих эффективных стратегиях и сможем дать рекомендации по выбору стратегии и её построению.

<center><p style = "background-color : white; border-radius: 2px 2px; padding : 3px"><strong>
<span style="color:#000">Risk / performance ratios (Соотношения риска и доходности)</span>

In [92]:
df_ratio = data[['title','Sharpe ratio', 'Sortino ratio', 'Profit factor', 'Margin calls']]

In [93]:
df_ratio

,title,Sharpe ratio,Sortino ratio,Profit factor,Margin calls
0,Daily Breakout + Daily Shadow By Rouro,0.203,0.453,6.241,0.0
1,Order Block Strategy,0.616,9.392,"1,426,164,099.778",0.0
2,EXODUS,−1.136,−0.782,1.262,0.0
5,Arrow's Flexible MA Cross Strategy [API Ready],0.368,0.994,2.046,0.0
6,1h Liquidity Swings Strategy with 1:2 RR,0.146,0.27,1.158,0.0
...,...,...,...,...,...
953,3Commas Bot DCA Backtester & Signals FREE,0.521,21.632,1.806,0.0
954,rt maax EMA cross strategy,0.024,2.104,25.843,0.0
956,Multi Trend Cross Strategy Template,0.137,1.885,9.789,0.0
957,BOLLY Bands,1.367,443.519,3.246,0.0


Надо числовые колонки привести подобающий вид.
Колонки `['title', 'Margin calls']` - останутся как есть

In [94]:
df_ratio[['Sharpe ratio', 'Sortino ratio', 'Profit factor']] = df_ratio[['Sharpe ratio', 'Sortino ratio', 'Profit factor']].map(convert_to_numeric)

C:\Users\xndrf\AppData\Local\Temp\ipykernel_3488\2312526999.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [95]:
df_ratio.describe()

,Sharpe ratio,Sortino ratio,Profit factor,Margin calls
count,8.290000e+02,829.000000,8.290000e+02,829.000000
mean,-1.698904e+05,5.168713,1.720356e+06,2.311218
std,4.847815e+06,38.911370,4.953273e+07,27.726531
min,-1.395796e+08,-1.000000,1.880000e-01,0.000000
25%,-2.760000e-01,-0.351000,1.200000e+00,0.000000
50%,1.310000e-01,0.291000,1.595000e+00,0.000000
75%,3.230000e-01,1.156000,2.312000e+00,0.000000
max,1.583000e+00,637.510000,1.426164e+09,567.000000


<div class="alert alert-danger">
Анализируя статистические показатели Коэффициента Шарпа, Сортино и профит фактора, можно сказать что в данных присутвуют Аномально высокие значения стандартного отклонения и отрицательные минимумы указывают на наличие серьезных проблем с данными или расчетами. Надо либо исключать эти стратегии из анализа, либо проверить корректность работы и расчёта формул на сайте по этим коэффициентам.


**В связи с этим, выполним работу по чистке стратегий с аномальными значениями, не смотря на то, что для этих коэффициентов нормальные диапазоны находятся в пределах от -1 до 2. Мы сделаем скидку и оставим стратегии с пределами коэффициентов от -3 до 3**

In [96]:
df_r = df_ratio[
    (df_ratio['Sharpe ratio'].between(-3, 3)) &
    (df_ratio['Sortino ratio'].between(-3, 3)) &
    (df_ratio['Profit factor'].between(-3, 3))
]

print(f'Количество аномальных стратегий: {df_ratio.shape[0]-df_r.shape[0]}')

Количество аномальных стратегий: 269


In [97]:
df_r['Margin calls'].value_counts()

Margin calls
0.0      550
1.0        3
320.0      1
290.0      1
567.0      1
28.0       1
209.0      1
7.0        1
38.0       1
Name: count, dtype: int64

<div class="alert alert-danger">


**В целом у Margin calls не особо много разных значений, в основном все нулевые, и я бы дропнул эту колонку и убрал с изучения**

<div class="alert alert-info">
    
Разберемся теперь с остальными коэффициентами и глянем, что из себя представляют

In [98]:
grap_list = ['Sharpe ratio', 'Sortino ratio', 'Profit factor']
for grap in grap_list:
    gist_graph(df_r, grap, grap)

<div class="alert alert-info">
    
Выделим топ 20 стратегий по каждому из показателей

In [99]:
sharpe = df_r.sort_values(by = 'Sharpe ratio', ascending=False).head(20)
sortino = df_r.sort_values(by = 'Sortino ratio', ascending=False).head(20)
profit = df_r.sort_values(by = 'Profit factor', ascending=False).head(20)


In [100]:
# Список новых датафреймов
list_df = [sharpe, sortino, profit]
# Название графиков
title_name = ['Sharpe ratio', 'Sortino ratio', 'Profit factor']
# Строим графики по новым датафреймам:
for list, name in zip(list_df, title_name):
    build_graph('px', 'bar', list, 'title', name, 20, 15, name, '', 'Коэффициент', color='title')

<div class="alert alert-info">
На графиках видно, что среди трех коэффициентов, предназначенных для оценки доходности инвестиций, есть пересечения между стратегиями. Но если рассматривать их в категории двух разных коэффициентов, в случае рассмотрения трех сразу у нас пересечений нет.
</div>

<div class="alert alert-danger">

**Возможно, дело в том, что мы делали скидку и оставили стратегии с пределами коэффициентов от -3 до 3. Возможно, стоит поработать с этим коэффициентом, и тогда найдется пересечение, или увеличить количество топовых стратегий по показателям с 20 до 100.**
</div>

<div class="alert alert-info">
Так как Коэффициент Шарпа и Коэффициент Сортино учитывают риски волатильности и оба измеряют доходность, то выделим на их основе самые топовые стратегии:
</div>


In [101]:
top_20_strategy = pd.merge(sharpe[['title']], sortino[['title']], on='title', how='inner')
top_20_strategy

,title
0,PSAR BBPT ZLSMA BTC 1min
1,Overnight Effect High Volatility Crypto (AiBit...
2,Dual Strategy Selector V2 - Cryptogyani
3,Bollinger Bands - Breakout Strategy
4,TrendGuard Scalper: SSL + Hama Candle with Co...
5,Strategy SEMA SDI Webhook
6,Ta Strategy
7,Post-Open Long Strategy with ATR-based Stop Lo...


<a id='1'></a>
<p style = "background-color : #79d1ff; border-radius: 5px 5px; padding : 10px"><strong>

# Performance (Отчет о стратегии)

<p style = "background-color : #79d1ff; border-radius: 5px 5px; padding : 10px"><strong>

<div class="alert alert-info">
    
Разберемся, что с пропусками
Для корректной работы необходимо провести корректировку и подготовку числовых данных. Поэтому создадим функцию с учетом особенностей наших колонок и переведем их в числовой вид:

In [102]:
# Для удобства создадим датасет с колонками по Отчетам о стратегии
df_p = df[['Open P&L', 'Net profit', 'Gross profit', 'Gross loss', 'Commission paid', 'Buy & hold return', 'Max equity run-up', 'Max equity drawdown', 'Max contracts held',]]

In [103]:
df_p.info()

<class 'pandas.core.frame.DataFrame'>
Index: 829 entries, 0 to 958
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Open P&L             829 non-null    object
 1   Net profit           829 non-null    object
 2   Gross profit         829 non-null    object
 3   Gross loss           829 non-null    object
 4   Commission paid      829 non-null    object
 5   Buy & hold return    829 non-null    object
 6   Max equity run-up    829 non-null    object
 7   Max equity drawdown  829 non-null    object
 8   Max contracts held   829 non-null    object
dtypes: object(9)
memory usage: 64.8+ KB


In [ ]:
# Приведем object в float
df_p = df_p.map(convert_to_numeric)
df_p

,Open P&L,Net profit,Gross profit,Gross loss,Commission paid,Buy & hold return,Max equity run-up,Max equity drawdown,Max contracts held
0,0.00,8375.00,9973.00,1.598000e+03,0.00,117304.17,8375.00,1608.50,100
1,-1453.09,9783.49,9783.49,1.000000e-05,0.00,-72.38,10667.89,3180.47,195976
2,0.00,14293.23,68836.40,5.454318e+04,1672.77,5593815.28,20726.51,7634.92,200
5,2.24,184.19,360.35,1.761500e+02,46.20,103.02,212.07,45.80,0
6,-1160.00,117390.00,859610.00,7.422200e+05,0.00,708340.00,155735.00,61280.00,1
...,...,...,...,...,...,...,...,...,...
953,-0.44,288.93,647.28,3.583500e+02,66.71,-1880.26,530.92,455.22,2473
954,39115.55,1066063.89,1108975.30,4.291141e+04,12611.77,9960471.92,1106625.50,23788.39,3778
956,0.00,28127665.94,31327842.85,3.200177e+06,125781.93,13648918.30,28133021.71,3116562.10,1969898
957,0.00,7998.34,11559.50,3.561160e+03,1448.00,3159.02,8014.34,720.72,432331
